In [1]:
from typing import Union

from datasets import Dataset, load_dataset
from peft import LoraConfig  # type: ignore
from pydantic import BaseModel, Field

import xverify as xv

In [2]:
class Tools(BaseModel):
    """
    Run a tool.
    """

    tool_use: xv.XMLToolUse[xv.calculator, xv.search] = Field(
        ..., description="The tool call to use"
    )


class FinalAnswer(BaseModel):
    """
    Return a final answer.
    """

    answer: int = Field(..., description="Final answer to the question")


class Reason_and_Act(BaseModel):
    scratchpad: str = Field(
        ...,
        description="Information from the Observation useful to answer the question",
    )
    reasoning: str = Field(
        ...,
        description="It describes your thoughts about the question you have been asked",
    )
    # response: Tools | FinalAnswer # this still doesn't work
    response: Union[Tools, FinalAnswer]


def tool_response_func(model: Reason_and_Act) -> dict | None:
    return xv.run_tools(model.response)


guided_schema = xv.GuidedSchema(
    Reason_and_Act,
    schema="xml",
    tool_response_func=tool_response_func,
)

In [3]:
def format_prompt(
    prompt: str, system_prompt: str | None = None
) -> list[dict[str, str]]:
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    return messages


def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

max_steps = 10

SYSTEM_PROMPT = f"""\
You are a helpful assistant, responding in XML structured output.

- Think step by step using the scratchpad and reasoning outputs. You have {max_steps - 1} steps to think before responding.
- Use the tools provided. DO NOT rely on your own knowledge when a tool is available to help you.
- Respond with a final answer only once your are absolutely sure you have the answer.

Respond with a XML object, following the schema below:

{guided_schema.doc}
"""

SYSTEM_PROMPT = """\
TEST PROMPT
"""

dataset: Dataset = load_dataset("openai/gsm8k", "main", split="train")  # type: ignore
dataset = dataset.map(lambda x: {
    'prompt': [
        {'role': 'system', 'content': SYSTEM_PROMPT},
        {'role': 'user', 'content': x['question']}
    ],
    'answer': extract_hash_answer(x['answer'])
})

In [4]:
messages = [{'content': 'TEST PROMPT\n', 'role': 'system'}, {'content': "There are 15 tables in the school's cafeteria. Each table can seat 10 people. Usually, only 1/10 of the seats are left unseated. How many seats are usually taken?", 'role': 'user'}, {'role': 'assistant', 'content': "<Reason_and_Act>\n<scratchpad>\n 1. Identify the total number of tables and the average number of seats per table.\n 2. Calculate the total number of seats in the cafeteria.\n 3. Determine the number of seats that are usually left unseated.\n 4. Subtraction the number of unseated seats from the total number of seats to find the number of seats usually taken.\n</scratchpad>\n<reasoning>\n 1. The total number of tables is 15 and each table can seat 10 people, so the total number of seats is 15 x 10 = 150.  \n 2. Usually, 1/10 of the seats are left unseated, so the number of seats usually left unseated is 150 x 1/10 = 15.  \n 3. The number of seats usually taken is the total number of seats minus the number of seats usually left unseated, so it is 150 - 15 = 135.  \n</reasoning>\n<response>\n<Tools>\n<tool_use>\n<search>\n<query>\nWhat is the number of seats usually taken in the school's cafeteria?\n</query>\n<num_results>\n150\n</num_results>\n</search>\n</tool_use>\n</Tools>\n</response>\n</Reason_and_Act>"}, {'role': 'user', 'content': {'tool_use': {'search': "• School Cafeteria Design and Best Practices\n  This article explores key aspects of the school cafeteria, including the challenges of catering to an international student body. ... consider the number of seats and the space needed for movement.\n\n• A school cafeteria has tables that can seat 12 students each. How many ...\n  Calculate the total number of tables needed. Since the 20 tables can seat 240 students, and there are 10 students left, you need one more table for those 10 students.\n\n• A Guide To Cafeteria/Lunchroom Seating In Schools\n  In high school, the cafeteria often serves as a study hall or after-hours meeting place for students.\n\n• Cafeteria Design Guide - Architecture Student Chronicles\n  Here is the list of some of the important spaces of a cafeteria that you might see around: Seating area - enclosed space; Seating area - semi-enclosed space; ... Seating area = 6m x 10m = 60sqm.\n\n• a school has 17 tables in the cafeteria. each table seats 12 students ...\n  And of course, its all-white public schools.\n\n• [FREE] A school has 17 tables in the cafeteria. Each table seats 12 ...\n  The school has a total of 17 tables. Each table can seat up to 12 students.\n\n• Should Students Have Assigned Seats in the Cafeteria?\n  Schools have started assigning seats in the cafeteria to curb the chaos and ensure every student is well-fed. But does this truly help in behavior management, or does it rain on students' parades?.\n\n• What is the cafeteria capacity? The concept design does not ... - Nashua\n  Using the requirements set forth by the 2015 International Building Code, the concept design for the Cafeteria would include 15 square feet per student in an arrangement of tables and chairs.\n\n• School Cafeteria Seating Chart - Teachers Printables\n  School Cafeteria Seating Chart.\n\n• School Cafeteria Seating Guide | SchoolOutlet.com\n  297 Kingsbury Grade Suite 133 Mailing: PO Box 4470 Stateline, NV 89449-4470."}}}, {'role': 'assistant', 'content': "<Reason_and_Act>\n<scratchpad>\n 1. Identify the total number of tables and the average number of seats per table.\n 2. Calculate the total number of seats in the cafeteria.\n 3. Determine the number of seats that are usually left unseated.\n 4. Subtraction the number of unseated seats from the total number of seats to find the number of seats usually taken.\n</scratchpad>\n<reasoning>\n 1. The total number of tables is 15 and each table can seat 10 people, so the total number of seats is 15 x 10 = 150.  \n 2. Usually, 1/10 of the seats are left unseated, so the number of seats usually left unseated is 150 x 1/10 = 15.  \n 3. The number of seats usually taken is the total number of seats minus the number of seats usually left unseated, so it is 150 - 15 = 135.  \n</reasoning>\n<response>\n<Tools>\n<tool_use>\n<search>\n<query>\nWhat is the number of seats usually taken in the school's cafeteria?\n</query>\n<num_results>\n150\n</num_results>\n</search>\n</tool_use>\n</Tools>\n</response>\n</Reason_and_Act>"}, {'role': 'user', 'content': {'tool_use': {'search': "• A school cafeteria has tables that can seat 12 students each. How many ...\n  Calculate the total number of tables needed. Since the 20 tables can seat 240 students, and there are 10 students left, you need one more table for those 10 students.\n\n• School Cafeteria Design and Best Practices\n  This article explores key aspects of the school cafeteria, including the challenges of catering to an international student body. ... consider the number of seats and the space needed for movement.\n\n• A Guide To Cafeteria/Lunchroom Seating In Schools\n  In high school, the cafeteria often serves as a study hall or after-hours meeting place for students.\n\n• in the school cafeteria there are there are 20 seats each table. How\n  Since there are 20 seats at each table, we can divide the total number of students by the number of seats at each table to determine the number of tables needed.\n\n• Cafeteria Design Guide - Architecture Student Chronicles\n  Here is the list of some of the important spaces of a cafeteria that you might see around: Seating area - enclosed space; Seating area - semi-enclosed space; ... Seating area = 6m x 10m = 60sqm.\n\n• School Cafeteria Seating Guide | SchoolOutlet.com\n  297 Kingsbury Grade Suite 133 Mailing: PO Box 4470 Stateline, NV 89449-4470.\n\n• School Cafeteria Seating Chart - Teachers Printables\n  School Cafeteria Seating Chart.\n\n• Should Students Have Assigned Seats in the Cafeteria?\n  Schools have started assigning seats in the cafeteria to curb the chaos and ensure every student is well-fed. But does this truly help in behavior management, or does it rain on students' parades?.\n\n• Design Considerations for School Cafeterias - TlwaStoria\n  Designing a school cafeteria involves numerous considerations to create an atmosphere that is welcoming, efficient, and conducive to the needs of students and staff alike.\n\n• there are 141 students who need to be seated in the cafeteria. each ...\n  There are 141 students who need to be seated in the cafeteria. Each table has 8 seats."}}}]
completion_mask = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
completion_ids = [27, 25139, 8378, 46672, 397, 27, 54055, 13242, 397, 220, 16, 13, 64547, 279, 2790, 1372, 315, 12632, 323, 279, 5461, 1372, 315, 16312, 817, 1965, 624, 220, 17, 13, 20517, 279, 2790, 1372, 315, 16312, 304, 279, 93848, 624, 220, 18, 13, 29901, 279, 1372, 315, 16312, 429, 525, 5990, 2115, 650, 325, 657, 624, 220, 19, 13, 3719, 26425, 279, 1372, 315, 650, 325, 657, 16312, 504, 279, 2790, 1372, 315, 16312, 311, 1477, 279, 1372, 315, 16312, 5990, 4429, 624, 522, 54055, 13242, 397, 27, 19895, 287, 397, 220, 16, 13, 576, 2790, 1372, 315, 12632, 374, 220, 16, 20, 323, 1817, 1965, 646, 10723, 220, 16, 15, 1251, 11, 773, 279, 2790, 1372, 315, 16312, 374, 220, 16, 20, 856, 220, 16, 15, 284, 220, 16, 20, 15, 13, 2303, 220, 17, 13, 32967, 11, 220, 16, 14, 16, 15, 315, 279, 16312, 525, 2115, 650, 325, 657, 11, 773, 279, 1372, 315, 16312, 5990, 2115, 650, 325, 657, 374, 220, 16, 20, 15, 856, 220, 16, 14, 16, 15, 284, 220, 16, 20, 13, 2303, 220, 18, 13, 576, 1372, 315, 16312, 5990, 4429, 374, 279, 2790, 1372, 315, 16312, 27283, 279, 1372, 315, 16312, 5990, 2115, 650, 325, 657, 11, 773, 432, 374, 220, 16, 20, 15, 481, 220, 16, 20, 284, 220, 16, 18, 20, 13, 2303, 522, 19895, 287, 397, 27, 2322, 397, 27, 16583, 397, 27, 14172, 15951, 397, 27, 1836, 397, 27, 1631, 397, 3838, 374, 279, 1372, 315, 16312, 5990, 4429, 304, 279, 2906, 594, 93848, 5267, 522, 1631, 397, 59327, 13576, 397, 16, 20, 15, 198, 522, 2413, 13576, 397, 522, 1836, 397, 522, 14172, 15951, 397, 522, 16583, 397, 522, 2322, 397, 522, 25139, 8378, 46672, 29, 151645, 198, 151644, 872, 198, 14172, 15951, 151645, 198, 151644, 77091, 198, 27, 25139, 8378, 46672, 397, 27, 54055, 13242, 397, 220, 16, 13, 64547, 279, 2790, 1372, 315, 12632, 323, 279, 5461, 1372, 315, 16312, 817, 1965, 624, 220, 17, 13, 20517, 279, 2790, 1372, 315, 16312, 304, 279, 93848, 624, 220, 18, 13, 29901, 279, 1372, 315, 16312, 429, 525, 5990, 2115, 650, 325, 657, 624, 220, 19, 13, 3719, 26425, 279, 1372, 315, 650, 325, 657, 16312, 504, 279, 2790, 1372, 315, 16312, 311, 1477, 279, 1372, 315, 16312, 5990, 4429, 624, 522, 54055, 13242, 397, 27, 19895, 287, 397, 220, 16, 13, 576, 2790, 1372, 315, 12632, 374, 220, 16, 20, 323, 1817, 1965, 646, 10723, 220, 16, 15, 1251, 11, 773, 279, 2790, 1372, 315, 16312, 374, 220, 16, 20, 856, 220, 16, 15, 284, 220, 16, 20, 15, 13, 2303, 220, 17, 13, 32967, 11, 220, 16, 14, 16, 15, 315, 279, 16312, 525, 2115, 650, 325, 657, 11, 773, 279, 1372, 315, 16312, 5990, 2115, 650, 325, 657, 374, 220, 16, 20, 15, 856, 220, 16, 14, 16, 15, 284, 220, 16, 20, 13, 2303, 220, 18, 13, 576, 1372, 315, 16312, 5990, 4429, 374, 279, 2790, 1372, 315, 16312, 27283, 279, 1372, 315, 16312, 5990, 2115, 650, 325, 657, 11, 773, 432, 374, 220, 16, 20, 15, 481, 220, 16, 20, 284, 220, 16, 18, 20, 13, 2303, 522, 19895, 287, 397, 27, 2322, 397, 27, 16583, 397, 27, 14172, 15951, 397, 27, 1836, 397, 27, 1631, 397, 3838, 374, 279, 1372, 315, 16312, 5990, 4429, 304, 279, 2906, 594, 93848, 5267, 522, 1631, 397, 59327, 13576, 397, 16, 20, 15, 198, 522, 2413, 13576, 397, 522, 1836, 397, 522, 14172, 15951, 397, 522, 16583, 397, 522, 2322, 29, 198, 522, 25139, 8378, 46672, 29, 151645]

In [5]:
len(completion_mask)

622

In [6]:
len(completion_ids)

594

In [7]:
messages

[{'content': 'TEST PROMPT\n', 'role': 'system'},
 {'content': "There are 15 tables in the school's cafeteria. Each table can seat 10 people. Usually, only 1/10 of the seats are left unseated. How many seats are usually taken?",
  'role': 'user'},
 {'role': 'assistant',
  'content': "<Reason_and_Act>\n<scratchpad>\n 1. Identify the total number of tables and the average number of seats per table.\n 2. Calculate the total number of seats in the cafeteria.\n 3. Determine the number of seats that are usually left unseated.\n 4. Subtraction the number of unseated seats from the total number of seats to find the number of seats usually taken.\n</scratchpad>\n<reasoning>\n 1. The total number of tables is 15 and each table can seat 10 people, so the total number of seats is 15 x 10 = 150.  \n 2. Usually, 1/10 of the seats are left unseated, so the number of seats usually left unseated is 150 x 1/10 = 15.  \n 3. The number of seats usually taken is the total number of seats minus the number of

In [8]:
def exact_answer_reward_func(completions, answer, **_) -> list[float]:
    """Reward function that checks if the final answer matches the expected answer."""

    def _check_answer(trajectory: list[dict[str, str]], answer: int) -> float:
        """Extract the last answer from a trajectory."""
        last_message = trajectory[-1]
        assert last_message["role"] == "assistant", "should be assistant"
        parsed: Reason_and_Act | None = guided_schema.parse(last_message["content"])  # type: ignore
        if parsed is None or not isinstance(parsed.response, FinalAnswer):
            return 0.0
        return 1.0 if parsed.response.answer == answer else 0.0

    return [_check_answer(c, a) for c, a in zip(completions, answer)]


model_name = "Qwen/Qwen2.5-0.5B-Instruct"
model, tokenizer = xv.get_model_and_tokenizer(model_name)


run_name = "gsm8k-calculator-peft_" + model_name.split("/")[-1].lower()
training_args = xv.get_default_grpo_config(run_name, num_gpus=1)

peft_config = LoraConfig(
    r=16,
    lora_alpha=64,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj"],
)


trainer = xv.GRPOGuidedTrainer(
    guided_schema=guided_schema,
    model=model,
    args=training_args,
    train_dataset=dataset,
    peft_config=peft_config,
    reward_funcs=[exact_answer_reward_func],
)
trainer.train()

Using Liger kernel
Applied Liger kernels to Qwen2


You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/home/tom/fun/xverify/.venv/lib/python3.12/site-packages/trl/trainer/grpo_trainer.py:471: UserWarning: The requested device cuda:0 is also being used for training. For higher throughput and to avoid out-of-memory errors, it is recommended to use a dedicated device for vLLM. If this is intentional, you may ignore this warning but should adjust `vllm_gpu_memory_utilization` accordingly.
  warnings.warn(


INFO 03-13 15:31:30 __init__.py:207] Automatically detected platform cuda.
INFO 03-13 15:31:35 config.py:549] This model supports multiple tasks: {'score', 'classify', 'generate', 'reward', 'embed'}. Defaulting to 'generate'.
INFO 03-13 15:31:35 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='Qwen/Qwen2.5-0.5B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-0.5B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda:0, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_e

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 03-13 15:31:37 model_runner.py:1115] Loading model weights took 0.9266 GB
INFO 03-13 15:31:38 worker.py:267] Memory profiling takes 0.84 seconds
INFO 03-13 15:31:38 worker.py:267] the current vLLM instance can use total_gpu_memory (23.58GiB) x gpu_memory_utilization (0.30) = 7.07GiB
INFO 03-13 15:31:38 worker.py:267] model weights take 0.93GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 1.44GiB; the rest of the memory reserved for KV Cache is 4.65GiB.
INFO 03-13 15:31:38 executor_base.py:111] # cuda blocks: 25399, # CPU blocks: 21845
INFO 03-13 15:31:38 executor_base.py:116] Maximum concurrency for 32768 tokens per request: 12.40x
INFO 03-13 15:31:42 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_ut

...

[2025-03-13 15:32:23] INFO lib.rs:450: response: https://html.duckduckgo.com/html 200
[2025-03-13 15:32:24] INFO lib.rs:450: response: https://html.duckduckgo.com/html 200
[2025-03-13 15:32:27] INFO lib.rs:450: response: https://lite.duckduckgo.com/lite/ 200


╭──────────────────────────────────────────────────── Step 0 ─────────────────────────────────────────────────────╮
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ │
│ ┃ Prompt                                          ┃ Completion                                       ┃ Reward ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ │
│ │ There are 15 tables in the school's cafeteria.  │ assistant: <Reason_and_Act>                      │   0.00 │ │
│ │ Each table can seat 10 people. Usually, only    │ <scratchpad>                                     │        │ │
│ │ 1/10 of the seats are left unseated. How many   │  1. Identify the total number of tables and the  │        │ │
│ │ seats are usually taken?                        │ average number of seats per table.               │        │ │
│ │                                                 │  2. Calculate the total number of seats in the   │        │ │
│ │                                                 │ cafeteria.                                       │        │ │
│ │                                                 │  3. Determine the number of seats that are       │        │ │
│ │                                                 │ usually left unseated.                           │        │ │
│ │                                                 │  4. Subtraction the number of unseated seats     │        │ │
│ │                                                 │ from the total number of seats to find the       │        │ │
│ │                                                 │ number of seats usually taken.                   │        │ │
│ │                                                 │ </scratchpad>                                    │        │ │
│ │                                                 │ <reasoning>                                      │        │ │
│ │                                                 │  1. The total number of tables is 15 and each    │        │ │
│ │                                                 │ table can seat 10 people, so the total number of │        │ │
│ │                                                 │ seats is 15 x 10 = 150.                          │        │ │
│ │                                                 │  2. Usually, 1/10 of the seats are left          │        │ │
│ │                                                 │ unseated, so the number of seats usually left    │        │ │
│ │                                                 │ unseated is 150 x 1/10 = 15.                     │        │ │
│ │                                                 │  3. The number of seats usually taken is the     │        │ │
│ │                                                 │ total number of seats minus the number of seats  │        │ │
│ │                                                 │ usually left unseated, so it is 150 - 15 = 135.  │        │ │
│ │                                                 │ </reasoning>                                     │        │ │
│ │                                                 │ <response>                                       │        │ │
│ │                                                 │ <Tools>                                          │        │ │
│ │                                                 │ <tool_use>                                       │        │ │
│ │                                                 │ <search>                                         │        │ │
│ │                                                 │ <query>                                          │        │ │
│ │                                                 │ What is the number of seats usually taken in the │        │ │
│ │                                                 │ school's cafeteria?                              │        │ │
│ │                                                 │ </

╭──────────────────────────────────────────────────── Step 0 ─────────────────────────────────────────────────────╮
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ │
│ ┃ Prompt                                          ┃ Completion                                       ┃ Reward ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ │
│ │ Fiona completed 36 math questions in an hour.   │ assistant: <Reason_and_Act>                      │   0.00 │ │
│ │ Shirley was able to complete twice as many math │ <scratchpad>                                     │        │ │
│ │ questions within that same time, and Kiana      │ •                                                │        │ │
│ │ completed half of the sum of Fiona and          │                                                  │        │ │
│ │ Shirley's math questions. If they each did the  │ To find the total number of math questions       │        │ │
│ │ same number of questions the following hour,    │ completed by Fiona, Shirley, and Kiana, we start │        │ │
│ │ how many math questions did all three girls     │ by calculating how many questions each girl      │        │ │
│ │ complete in 2 hours?                            │ completed in the first hour.                     │        │ │
│ │                                                 │ •                                                │        │ │
│ │                                                 │                                                  │        │ │
│ │                                                 │ Fiona completed `36` questions in the first      │        │ │
│ │                                                 │ hour.                                            │        │ │
│ │                                                 │ •                                                │        │ │
│ │                                                 │                                                  │        │ │
│ │                                                 │ Shirley completed `2 * 36` questions in the      │        │ │
│ │                                                 │ first hour.                                      │        │ │
│ │                                                 │ •                                                │        │ │
│ │                                                 │                                                  │        │ │
│ │                                                 │ Kiana completed half of the sum of Fiona's and   │        │ │
│ │                                                 │ Shirley's questions in the first hour.           │        │ │
│ │                                                 │ •                                                │        │ │
│ │                                                 │                                                  │        │ │
│ │                                                 │ Kiana completed `0.5 * (36 + 2 * 36)` questions  │        │ │
│ │                                                 │ in the first hour.                               │        │ │
│ │                                                 │ •                                                │        │ │
│ │                                                 │                                                  │        │ │
│ │                                                 │ Next, to find out how many questions they each   │        │ │
│ │                                                 │ completed in the second hour, we multiply the    │        │ │
│ │                                                 │ number of questions each completed in an hour by │        │ │
│ │                                                 │ `2`.                                             │        │ │
│ │                                                 │ • 

[{'content': 'TEST PROMPT\n', 'role': 'system'}, {'content': 'In a graveyard, there are 20 skeletons.  Half of these skeletons are adult women, and the remaining number are split evenly between adult men and children.  If an adult woman has 20 bones in their body, and a male has 5 more than this, and a child has half as many as an adult woman, how many bones are in the graveyard?', 'role': 'user'}, {'role': 'assistant', 'content': '<Reason_and_Act>\n<scratchpad>\nThe total number of bones in the graveyard is 300.\n</scratchpad>\n<reasoning>\n1. Adult women have 20 bones each.\n2. There are half of the 20 bones attributed to adult women, which is 100 bones in total.\n3. The remaining, adult men and children, are split evenly between man and woman. Therefore, there would be 100 bones attributed to adult men as well.\n4. An adult man has 5 more bones than an adult woman, meaning each adult man has 25 bones.\n5. There are 100 bones attributed to adult men, so the total number of bones attr

ValueError: Completion mask and completion ids are not the same length for state 1